# 1) Extraindo os dados dos arquivos PDF
**Objetivo:** extrair texto de PDFs (Estatística/Política Monetária mensal, Atas do COPOM, REF semestral), limpar ruído (cabeçalho/rodapé, títulos, notas, números de página), consolidar **1 linha = 1 documento** e salvar a base para etapas futuras (ex.: sentimento por documento).

**Escopo desta versão:** _não_ calcula índice de sentimento — apenas prepara a base limpa e consolidada.

---

## Estrutura
1. Instalação de dependências (opcional)
2. Configuração
3. Utilidades de limpeza (cleaners)
4. Extração página a página (com fallbacks)
5. Parsing por documento (data, parágrafos)
6. Pipeline (varredura de pastas, cache, consolidação por documento)
7. Execução (gerar CSVs) + checagens rápidas

In [1]:
%pip install -q pdfplumber==0.11.0 pypdf==4.3.1


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ## 2) Configuração

from pathlib import Path

# Pastas com PDFs
ROOTS = [
    "data/estatisticas",
    "data/atas",
    "data/ref",
]

# Anos a considerar
YEARS = {2022, 2023, 2024, 2025}

# Parâmetros
MIN_WORDS = 8
CACHE_DIR = "out/pdf_paragrafos"   # onde ficam os .pkl por doc
REBUILD_CACHE = False              # True apenas quando mudar as regras ou na 1ª execução
TIPOS = {"mensal", "ata", "ref"}   # filtros de tipo (use None para não filtrar)


In [3]:
# ## 3) Utilidades de limpeza (cleaners)

from __future__ import annotations
import re, unicodedata
from typing import List, Set, Tuple, Optional
import pandas as pd
from pypdf import PdfReader

try:
    import pdfplumber
except Exception:
    pdfplumber = None

try:
    from pdfminer_high_level import extract_text as pdfminer_extract_text  # se já estava importando de pdfminer.high_level, mantenha lá
except Exception:
    try:
        from pdfminer.high_level import extract_text as pdfminer_extract_text
    except Exception:
        pdfminer_extract_text = None

# Padrões para remoção
PAGE_RE = re.compile(r'^\s*(p[aá]gina)\s+\d+(\s+de\s+\d+)?\s*$', re.I)
ONLY_DIGITS_RE = re.compile(r'^\s*\d+\s*$')
CAPTION_RE = re.compile(r'^\s*(figura|gr[aá]fico|tabela)\s*\d+[\.:].*$', re.I)
NOTE_PREFIX_RE = re.compile(r'^\s*(nota[s]?|obs\.?|observa[cç][aã]o|fonte|elabora[cç][aã]o)\s*[:\-–].*$', re.I)
FOOTNOTE_MARK_RE = re.compile(r'\[\d+\]')
SUPERSCRIPTS_RE = re.compile(r'[\u00B9\u00B2\u00B3\u2070-\u2079]+')
STOP_AFTER_HEADINGS_RE = re.compile(r'^\s*(refer[eê]ncias|anexos?|gloss[aá]rio)\b.*$', re.I | re.M)

HEADER_HINTS = (
    "banco central do brasil",
    "comitê de política monetária",
    "relatório de estabilidade financeira",
    "ata do copom",
    "relatório de estatística",
    "política monetária",
)

DOC_TYPE_CONFIG = {
    "ata":    {"stop_after": STOP_AFTER_HEADINGS_RE},
    "ref":    {"stop_after": STOP_AFTER_HEADINGS_RE},
    "mensal": {"stop_after": STOP_AFTER_HEADINGS_RE},
}

def normalize_line(s: str) -> str:
    s = unicodedata.normalize("NFC", s).strip()
    return re.sub(r'\s+', ' ', s)

def is_short_heading(line: str, max_words: int = 8) -> bool:
    l = normalize_line(line)
    if not l:
        return False
    words = l.split()
    if len(words) > max_words:
        return False
    letters = [ch for ch in l if ch.isalpha()]
    if not letters:
        return False
    upper_ratio = sum(ch.isupper() for ch in letters) / len(letters)
    return upper_ratio >= 0.8

def join_hyphenated(text: str) -> str:
    return re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

def normalize_whitespace(text: str) -> str:
    text = text.replace("\r", "\n")
    text = re.sub(r'\n{3,}', '\n\n', text)         # >=3 quebras -> 2
    text = re.sub(r'[ \t]+', ' ', text)            # espaços múltiplos -> 1
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)   # quebra simples dentro de frase -> espaço
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\s+([,.;:!?])', r'\1', text)
    return re.sub(r'\n{3,}', '\n\n', text).strip()

def detect_repeating_edge_lines(pages_lines: List[List[str]],
                                top_n: int = 2, bottom_n: int = 2,
                                threshold: float = 0.5) -> Set[str]:
    from collections import Counter
    cand, total = Counter(), max(1, len(pages_lines))
    for lines in pages_lines:
        lines_norm = [normalize_line(x) for x in lines if x.strip()]
        top = lines_norm[:top_n]
        bottom = lines_norm[-bottom_n:] if bottom_n else []
        for item in set(top + bottom):
            if item:
                cand[item] += 1
    rep = {s for s, c in cand.items() if c / total >= threshold}
    rep |= {s for s in cand if any(h in s.lower() for h in HEADER_HINTS)}
    return rep

def remove_edge_lines(lines: List[str], repetitive: Set[str]) -> List[str]:
    out = []
    for ln in lines:
        n = normalize_line(ln)
        if not n:
            continue
        if n in repetitive:
            continue
        if PAGE_RE.match(n) or ONLY_DIGITS_RE.match(n):
            continue
        out.append(ln)
    return out

def clean_page_text(raw: str, repetitive: Set[str]) -> str:
    lines = remove_edge_lines(raw.splitlines(), repetitive)
    kept = []
    for ln in lines:
        l = normalize_line(ln)
        if not l:
            continue
        if is_short_heading(l):
            continue
        if NOTE_PREFIX_RE.match(l) or CAPTION_RE.match(l):
            continue
        l = FOOTNOTE_MARK_RE.sub('', l)
        l = SUPERSCRIPTS_RE.sub('', l)
        kept.append(l)
    return "\n".join(kept).strip()

def apply_stop_after(text: str, doc_type: str) -> str:
    pat = DOC_TYPE_CONFIG.get(doc_type, {}).get("stop_after")
    if not pat:
        return text
    m = pat.search(text)
    return text[:m.start()].rstrip() if m else text

def guess_doc_type(name: str, path: Optional[Path] = None) -> str:
    """
    Classifica o PDF em: 'ata' (Copom), 'ref' (Relatório de Estabilidade Financeira)
    ou 'mensal' (Estatística/Política Monetária).
    Regras focadas nos padrões:
      - Copom249-not20220921249.pdf  -> ata
      - RELESTAB202204-refPub.pdf    -> ref
    """
    s = (name or "").lower()
    p = (str(path.parent).lower() if path else "")
    hay = f"{s} {p}"

    # --- Atas do Copom ---
    if (
        "copom" in hay or                      # Copom249-not20220921249.pdf
        re.search(r'(^|[/_\-\s])ata([/_\-\s]|$)', hay) or  # .../atas/... ou arquivos com "ata"
        "atas" in hay
    ):
        return "ata"

    # --- Relatório de Estabilidade Financeira (REF) ---
    if (
        "relestab" in hay or                   # RELESTAB202204-refPub.pdf
        "refpub" in hay or
        re.search(r'(^|[/_\-\s])ref([/_\-\s]|$)', hay) or
        "estabilidade" in hay
    ):
        return "ref"

    # --- Relatório mensal de estatística/política monetária ---
    if any(k in hay for k in [
        "estatistica", "estatísticas", "politica monetaria", "política monetária",
        "credito", "crédito"
    ]):
        return "mensal"

    return "mensal"


In [4]:
# ## 4) Extração página a página (com fallbacks)

def extract_pages_text(path_pdf: Path) -> List[str]:
    pages: List[str] = []
    # 1) pdfplumber (preferido)
    if pdfplumber is not None:
        try:
            with pdfplumber.open(str(path_pdf)) as pdf:
                for page in pdf.pages:
                    t = page.extract_text() or ""
                    pages.append(t)
        except Exception:
            pages = []
    # 2) pypdf (fallback)
    if not pages:
        try:
            reader = PdfReader(str(path_pdf))
            pages = [(p.extract_text() or "") for p in reader.pages]
        except Exception:
            pages = []
    # 3) pdfminer (fallback final)
    if (not pages or not any(x.strip() for x in pages)) and pdfminer_extract_text:
        try:
            txt = pdfminer_extract_text(str(path_pdf)) or ""
            pages = [p for p in re.split(r'\\f+', txt)]
        except Exception:
            pages = []
    return pages


In [5]:
# ## 5) Parsing por documento (data e parágrafos)

def clean_pdf_text(path_pdf: Path, doc_type: str) -> Tuple[str, Set[str], int]:
    raw_pages = extract_pages_text(path_pdf)
    if not raw_pages or not any(p.strip() for p in raw_pages):
        return "", set(), 0

    pages_lines = [p.splitlines() for p in raw_pages]
    repetitive = detect_repeating_edge_lines(pages_lines, top_n=2, bottom_n=2, threshold=0.5)
    cleaned_pages = [clean_page_text(p, repetitive) for p in raw_pages]

    text = "\n\n".join(p for p in cleaned_pages if p.strip())
    text = join_hyphenated(text)
    text = normalize_whitespace(text)
    text = apply_stop_after(text, doc_type)
    return text, repetitive, len(raw_pages)


def quebrar_paragrafos(txt: str, min_palavras: int = 8) -> list[str]:
    brutos = [p.strip() for p in re.split(r"\n\s*\n", txt) if p.strip()]
    pars = []
    for p in brutos:
        p = " ".join([l.strip() for l in p.split("\n") if l.strip()])
        if len(p.split()) >= min_palavras:
            pars.append(p)
    return pars


from datetime import datetime
import re

def _detect_date_from_name_and_path(path: Path) -> tuple[str, pd.Timestamp] | None:
    name = path.stem

    # 1) YYYY[-_.]?MM[-_.]?DD (com separadores opcionais)
    m = re.search(r'(20\d{2})[-_.]?([01]\d)[-_.]?([0-3]\d)', name)
    if m:
        y, mm, dd = int(m.group(1)), int(m.group(2)), int(m.group(3))
        try:
            dt = datetime(y, mm, dd)
            return f"{y}{mm:02d}", pd.Timestamp(dt)
        except ValueError:
            pass

    # 2) 8 dígitos contíguos começando com 20: YYYYMMDD
    for m in re.finditer(r'(20\d{6})', name):
        s = m.group(1)
        y, mm, dd = int(s[:4]), int(s[4:6]), int(s[6:8])
        try:
            dt = datetime(y, mm, dd)
            return f"{y}{mm:02d}", pd.Timestamp(dt)
        except ValueError:
            continue

    # 3) 6 dígitos contíguos começando com 20: YYYYMM
    for m in re.finditer(r'(20\d{4})', name):
        s = m.group(1)
        y, mm = int(s[:4]), int(s[4:6])
        if 1 <= mm <= 12:
            return s, pd.Timestamp(y, mm, 1)

    # 4) Fallback: ano pela pasta (ex.: .../atas/2022/) + mês de 2 dígitos no nome
    pstr = str(path.parent).lower()
    my = re.search(r'(20\d{2})', pstr)
    mm = re.search(r'(?:^|[^0-9])(0[1-9]|1[0-2])(?:[^0-9]|$)', name)
    if my and mm:
        y = int(my.group(1)); m = int(mm.group(1))
        return f"{y}{m:02d}", pd.Timestamp(y, m, 1)

    return None

def data_do_arquivo(path_pdf: Path) -> tuple[str, pd.Timestamp]:
    out = _detect_date_from_name_and_path(path_pdf)
    if out is None:
        raise ValueError(f"Não encontrei data no nome/pasta do arquivo: {path_pdf.name}")
    return out


In [10]:
# ## 6) Pipeline — MODO DOCUMENTO (1 linha = 1 PDF)

import re
import pandas as pd
from pathlib import Path
from typing import Iterable, Union

# --- helpers para nome canônico ---
TIPO_ALIAS_OUT = {"mensal": "estm", "ata": "copom", "ref": "ref"}

def doc_canonico(doc_src: str) -> str:
    """
    Converte 'YYYYMM_tipo' em 'YYYYMM_estm|copom|ref'
    Ex.: 202203_ata -> 202203_copom
    """
    yyyymm = doc_src[:6]
    tipo = doc_src.rsplit("_", 1)[-1].lower()
    return f"{yyyymm}_{TIPO_ALIAS_OUT.get(tipo, tipo)}"

# >>> pressupõe que estas funções já existem em células anteriores:
# data_do_arquivo(path_pdf: Path) -> (yyyymm, pd.Timestamp)
# guess_doc_type(name: str, path: Path|None) -> "mensal"|"ata"|"ref"
# clean_pdf_text(path_pdf: Path, doc_type: str) -> (texto, removed, n_pages)

def processar_pdf_doc(path_pdf: Path, yyyymm_override: str | None = None) -> dict:
    # meta
    yyyymm, dt = data_do_arquivo(path_pdf)
    if yyyymm_override:
        yyyymm = yyyymm_override
    tipo = guess_doc_type(path_pdf.name, path=path_pdf)
    doc_src = f"{yyyymm}_{tipo}"

    # texto limpo (documento inteiro)
    text, _removed, _n_pages = clean_pdf_text(path_pdf, tipo)
    if not text.strip():
        return {}  # pular doc vazio

    # nome canônico (YYYYMM_estm|copom|ref)
    doc = doc_canonico(doc_src)
    return {
        "doc": doc,
        "doc_src": doc_src,
        "yyyymm": yyyymm,
        "tipo": tipo,
        "dt": dt,
        "n_chars": len(text),
        "n_palavras": len(text.split()),
        "texto": text,
    }

def processar_raiz_docs(raiz: Union[str, Path, Iterable[Union[str, Path]]],
                        anos: set[int] | None = None,
                        rebuild_cache: bool = False,
                        cache_dir: str = "out/pdf_docs") -> pd.DataFrame:
    """
    Varre 1 ou N pastas, gera diretamente df_docs (1 linha = 1 PDF).
    Usa cache por documento (YYYYMM_tipo) em `cache_dir`.
    """
    roots = [Path(raiz)] if isinstance(raiz, (str, Path)) else [Path(r) for r in raiz]
    cache = Path(cache_dir)
    cache.mkdir(parents=True, exist_ok=True)

    rows, vistos = [], set()

    for root in roots:
        for pdf in sorted(root.rglob("*")):
            if not pdf.is_file() or pdf.suffix.lower() != ".pdf":
                continue

            name = pdf.stem
            # datas permissivas (pegam Copom249-not20220921249.pdf)
            m8 = re.search(r'(20\d{6})', name)   # YYYYMMDD
            m6 = re.search(r'(20\d{4})', name)   # YYYYMM
            if m8:
                yyyymm = m8.group(0)[:6]
            elif m6:
                yyyymm = m6.group(0)
            else:
                continue

            if anos is not None and int(yyyymm[:4]) not in anos:
                continue

            tipo = guess_doc_type(name, path=pdf)
            doc_key = f"{yyyymm}_{tipo}"
            if doc_key in vistos:
                continue

            pkl_path = cache / f"{doc_key}.pkl"
            if (not rebuild_cache) and pkl_path.exists():
                try:
                    row = pd.read_pickle(pkl_path)
                    if isinstance(row, dict) and row.get("texto"):
                        rows.append(row)
                        vistos.add(doc_key)
                        continue
                except Exception:
                    pass

            row = processar_pdf_doc(pdf, yyyymm_override=yyyymm)
            if row:
                pd.to_pickle(row, pkl_path)
                rows.append(row)
                vistos.add(doc_key)

    if not rows:
        return pd.DataFrame(columns=["doc","doc_src","yyyymm","tipo","dt","n_chars","n_palavras","texto"])

    df_docs = pd.DataFrame(rows).sort_values(["dt","doc"]).reset_index(drop=True)
    return df_docs


In [11]:
# ## 7) Execução (gerar CSVs) + checagens

df_docs = processar_raiz_docs(
    ["data/estatisticas", "data/atas", "data/ref"],
    anos={2022, 2023, 2024, 2025},
    rebuild_cache=True,           # True só nesta rodada
    cache_dir="out/pdf_docs",
)

assert df_docs["doc"].is_unique
print(df_docs["tipo"].value_counts())
df_docs.to_csv("textos_bc.csv")

# Visualização rápida
df_docs.head()


tipo
mensal    42
ata       27
Name: count, dtype: int64


,doc,doc_src,yyyymm,tipo,dt,n_chars,n_palavras,texto
0,202201_estm,202201_mensal,202201,mensal,2022-01-01,9189,1464,28.01.2022\n\n1. Crédito ampliado ao setor não...
1,202202_estm,202202_mensal,202202,mensal,2022-02-01,10180,1629,24.02.2022\n\n1. Crédito ampliado ao setor não...
2,202202_copom,202202_ata,202202,ata,2022-02-02,12107,1902,244ª Ata da Reunião do Comitê de Política Mone...
3,202203_estm,202203_mensal,202203,mensal,2022-03-01,7075,1141,28.04.2022\n\n1. Crédito ampliado ao setor não...
4,202203_copom,202203_ata,202203,ata,2022-03-16,15310,2449,245ª Ata da Reunião do Comitê de Política Mone...



> **Notas**
> - PDFs com layout multi-coluna podem exigir fallback adicional (`pdftotext -layout`) ou ajustes finos.
> - PDFs escaneados (imagem) precisam de OCR (ex.: `pytesseract` + `pdf2image`). Este pipeline não faz OCR.
> - Se alterar regras de limpeza (regex, `MIN_WORDS`, etc.), rode com `REBUILD_CACHE = True` uma vez.
> - A coluna `doc` já sai no formato **`YYYYMM_estm|copom|ref`** conforme o padrão decidido.

## Instalação

O que faz: instala libs para carregar modelos (transformers), datasets (datasets), métricas (sklearn) e treino (torch/accelerate).

In [12]:
pip install -U transformers datasets scikit-learn accelerate torch


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 3) FinBERT-PT-BR

In [13]:
import torch
import pandas as pd
from pathlib import Path
from transformers import (
    AutoTokenizer, 
    BertForSequenceClassification,
    pipeline,
)

# ——— FinBERT-PT-BR ———
device = "cuda" if torch.cuda.is_available() else "cpu"

finbert_pt_br_tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbert_pt_br_model = BertForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR").to(device).eval()

# Opcional / teste rápido (vai truncar textos longos — use só para frases curtas)
finbert_pt_br_pipeline = pipeline(
    task='text-classification',
    model=finbert_pt_br_model,
    tokenizer=finbert_pt_br_tokenizer,
    device=0 if device=="cuda" else -1
)

# Mapa id -> rótulo (padronizado em PT-BR)
id2label = finbert_pt_br_model.config.id2label
def _pt_label(i: int) -> str:
    l = str(id2label.get(i, "")).lower()
    if "pos" in l: return "positivo"
    if "neg" in l: return "negativo"
    if "neu" in l: return "neutro"
    if l.startswith("label_"):
        j = int(l.split("_")[-1])
        return _pt_label(j) if j in finbert_pt_br_model.config.id2label else "neutro"
    return "neutro" if not l else l


Device set to use cpu


In [14]:
import torch.nn.functional as F

MAX_LEN = 512
CHUNK_BODY = MAX_LEN - 2
MAX_CHUNKS = 20

def classify_text_finbert(texto: str) -> dict:
    texto = (texto or "").strip()
    if not texto:
        return {"label":"neutro","conf":1.0,"p_positivo":0.0,"p_negativo":0.0,"p_neutro":1.0}

    # 1) tokenize sem especiais p/ controlar cortes
    ids = finbert_pt_br_tokenizer.encode(texto, add_special_tokens=False)
    if not ids:
        return {"label":"neutro","conf":1.0,"p_positivo":0.0,"p_negativo":0.0,"p_neutro":1.0}

    # 2) corta em janelas
    chunks = [ids[i:i+CHUNK_BODY] for i in range(0, len(ids), CHUNK_BODY)]
    if len(chunks) > MAX_CHUNKS:
        chunks = chunks[:MAX_CHUNKS]

    # 3) para cada chunk, decodifica para TEXTO e usa o tokenizer (__call__)
    probs_sum = None
    for ch in chunks:
        chunk_text = finbert_pt_br_tokenizer.decode(ch, clean_up_tokenization_spaces=True)
        enc = finbert_pt_br_tokenizer(
            chunk_text,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_LEN
        )
        enc = {k: v.to(device) for k, v in enc.items()}
        with torch.no_grad():
            logits = finbert_pt_br_model(**enc).logits  # [1, C]
            p = F.softmax(logits, dim=-1).squeeze(0).cpu()
        probs_sum = p if probs_sum is None else (probs_sum + p)

    probs = probs_sum / len(chunks)

    id2label = finbert_pt_br_model.config.id2label
    def _pt(i): 
        l = str(id2label.get(i,"")).lower()
        if "pos" in l: return "positivo"
        if "neg" in l: return "negativo"
        if "neu" in l: return "neutro"
        return "neutro" if not l else l

    scores = {_pt(i): float(probs[i]) for i in range(probs.shape[-1])}
    p_pos = scores.get("positivo", 0.0)
    p_neg = scores.get("negativo", 0.0)
    p_neu = scores.get("neutro",   0.0)
    label = max(("positivo","negativo","neutro"), key=lambda k: {"positivo":p_pos,"negativo":p_neg,"neutro":p_neu}[k])
    conf = {"positivo":p_pos,"negativo":p_neg,"neutro":p_neu}[label]
    return {"label":label, "conf":conf, "p_positivo":p_pos, "p_negativo":p_neg, "p_neutro":p_neu}


In [15]:
# antes (no topo)
from datetime import datetime
from datetime import datetime, timezone


IN_CSV  = Path("textos_bc.csv")         # entrada (1 linha = 1 PDF)
OUT_CSV = Path("textos_com_sentimento.csv")  # saída (checkpoint)
CHECKPOINT_EVERY = 5                                 # salva a cada N docs

# 1) lê entrada
if not IN_CSV.exists():
    raise FileNotFoundError(f"Entrada não encontrada: {IN_CSV}")
df_docs = pd.read_csv(IN_CSV, encoding="utf-8-sig")

# chave para controle de progresso
KEY_COL = "doc_src" if "doc_src" in df_docs.columns else "doc"
if KEY_COL not in df_docs.columns:
    df_docs[KEY_COL] = df_docs["doc"]

# 2) carrega progresso já salvo e cria conjunto de já processados
processed_keys = set()
if OUT_CSV.exists():
    try:
        old = pd.read_csv(OUT_CSV, encoding="utf-8-sig")
        if KEY_COL in old.columns:
            processed_keys = set(old[KEY_COL].astype(str))
    except Exception:
        pass

print(f"Total a processar: {len(df_docs)} | Já processados: {len(processed_keys)}")

# 3) loop linha a linha com checkpoints
buffer = []
written_once = OUT_CSV.exists()
written_count = 0
meta_cols = [c for c in ["doc","doc_src","yyyymm","tipo","dt","n_chars","n_palavras"] if c in df_docs.columns]

for _, row in df_docs.iterrows():
    key = str(row[KEY_COL])
    if key in processed_keys:
        continue

    res = classify_text_finbert(str(row.get("texto", "")))

    out_row = {
        KEY_COL: key,
        "doc": row.get("doc", key),
        "sent_label": res["label"],
        "sent_conf":  res["conf"],
        "sent_p_positivo": res["p_positivo"],
        "sent_p_negativo": res["p_negativo"],
        "sent_p_neutro":   res["p_neutro"],
        "sent_model": "FinBERT-PT-BR",
        "processed_at": datetime.now(timezone.utc).isoformat(timespec="seconds").replace("+00:00", "Z"),
    }
    for c in meta_cols:
        out_row[c] = row.get(c, None)
    buffer.append(out_row)

    if len(buffer) >= CHECKPOINT_EVERY:
        pd.DataFrame(buffer).to_csv(
            OUT_CSV, mode="a", header=not written_once,
            index=False, encoding="utf-8-sig", lineterminator="\n"
        )
        written_once = True
        processed_keys.update([r[KEY_COL] for r in buffer])
        written_count += len(buffer)
        buffer = []
        print(f"Checkpoint: {written_count} novos documentos salvos...")

if buffer:
    pd.DataFrame(buffer).to_csv(
        OUT_CSV, mode="a", header=not written_once,
        index=False, encoding="utf-8-sig", lineterminator="\n"
    )
    processed_keys.update([r[KEY_COL] for r in buffer])
    written_count += len(buffer)
    print(f"Finalizado: {written_count} novos documentos salvos.")


Total a processar: 69 | Já processados: 0
Checkpoint: 5 novos documentos salvos...
Checkpoint: 10 novos documentos salvos...
Checkpoint: 15 novos documentos salvos...
Checkpoint: 20 novos documentos salvos...
Checkpoint: 25 novos documentos salvos...
Checkpoint: 30 novos documentos salvos...
Checkpoint: 35 novos documentos salvos...
Checkpoint: 40 novos documentos salvos...
Checkpoint: 45 novos documentos salvos...
Checkpoint: 50 novos documentos salvos...
Checkpoint: 55 novos documentos salvos...
Checkpoint: 60 novos documentos salvos...
Checkpoint: 65 novos documentos salvos...
Finalizado: 69 novos documentos salvos.


In [17]:
df = pd.read_csv("textos_com_sentimento.csv")
df.columns = [c.strip().lower() for c in df.columns]
df

,doc_src,doc,sent_label,sent_conf,sent_p_positivo,sent_p_negativo,sent_p_neutro,sent_model,processed_at,yyyymm,tipo,dt,n_chars,n_palavras
0,202201_mensal,202201_estm,positivo,0.419371,0.419371,0.283059,0.297571,FinBERT-PT-BR,2025-10-27T03:40:10Z,202201,mensal,2022-01-01,9189,1464
1,202202_mensal,202202_estm,negativo,0.387029,0.243111,0.387029,0.369860,FinBERT-PT-BR,2025-10-27T03:40:16Z,202202,mensal,2022-02-01,10180,1629
2,202202_ata,202202_copom,negativo,0.608266,0.069367,0.608266,0.322367,FinBERT-PT-BR,2025-10-27T03:40:22Z,202202,ata,2022-02-02,12107,1902
3,202203_mensal,202203_estm,positivo,0.585208,0.585208,0.224158,0.190634,FinBERT-PT-BR,2025-10-27T03:40:26Z,202203,mensal,2022-03-01,7075,1141
4,202203_ata,202203_copom,neutro,0.476025,0.087618,0.436357,0.476025,FinBERT-PT-BR,2025-10-27T03:40:33Z,202203,ata,2022-03-16,15310,2449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,202507_mensal,202507_estm,negativo,0.453914,0.323659,0.453914,0.222428,FinBERT-PT-BR,2025-10-27T03:48:36Z,202507,mensal,2025-07-01,11912,2045
65,202507_ata,202507_copom,negativo,0.495445,0.078152,0.495445,0.426403,FinBERT-PT-BR,2025-10-27T03:48:45Z,202507,ata,2025-07-30,17721,2782
66,202508_mensal,202508_estm,positivo,0.397181,0.397181,0.349911,0.252908,FinBERT-PT-BR,2025-10-27T03:48:52Z,202508,mensal,2025-08-01,10879,1881
67,202509_mensal,202509_estm,positivo,0.381392,0.381392,0.293765,0.324844,FinBERT-PT-BR,2025-10-27T03:48:59Z,202509,mensal,2025-09-01,12173,2016


In [18]:
# Normalização simples de rótulos
def norm_lbl(x: str) -> str:
    x = str(x).strip().lower()
    if "pos" in x: return "positivo"
    if "neg" in x: return "negativo"
    if "neu" in x: return "neutro"
    return x

if "sent_label" in df.columns:
    df["sent_label"] = df["sent_label"].map(norm_lbl)

# ======= OVERVIEW (não precisa de gold) =======
# 1) Distribuição de rótulos
dist_labels = (
    df["sent_label"].value_counts(dropna=False)
      .rename_axis("sent_label").reset_index(name="count")
)
display(dist_labels)

# 2) Distribuição por tipo × rótulo (se houver 'tipo')
if "tipo" in df.columns:
    by_tipo = (
        df.groupby(["tipo","sent_label"], dropna=False).size()
          .reset_index(name="count").sort_values(["tipo","count"], ascending=[True, False])
    )
    display(by_tipo)

# 3) Estatísticas de confiança por rótulo
if "sent_conf" in df.columns:
    df["sent_conf"] = pd.to_numeric(df["sent_conf"], errors="coerce").fillna(0.0)
else:
    df["sent_conf"] = 0.0
conf_stats = (
    df.groupby("sent_label", dropna=False)["sent_conf"]
      .agg(["count","mean","median","min","max"]).reset_index()
      .sort_values("count", ascending=False)
)
display(conf_stats)

# 4) Cobertura mensal × rótulo (se houver 'yyyymm')
if "yyyymm" in df.columns:
    by_month = (
        df.assign(yyyymm=df["yyyymm"].astype(str))
          .groupby(["yyyymm","sent_label"], dropna=False).size().reset_index(name="count")
          .sort_values(["yyyymm","sent_label"])
    )
    display(by_month)

# 5) 12 documentos com menor confiança (revisão)
if "texto" in df.columns:
    df["snippet"] = df["texto"].astype(str).str.replace(r"\s+", " ", regex=True).str.slice(0, 220) + "..."
else:
    df["snippet"] = ""
cols_low = [c for c in ["doc","doc_src","tipo","yyyymm","sent_label","sent_conf","snippet"] if c in df.columns]
display(df.sort_values("sent_conf").head(12)[cols_low])

# ======= MÉTRICAS (precisa de gold_label) =======
if "gold_label" in df.columns:
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, brier_score_loss
    df["gold_label"] = df["gold_label"].map(norm_lbl)
    eval_df = df[df["gold_label"].isin(["positivo","negativo","neutro"])].copy()

    if not eval_df.empty:
        y_true = eval_df["gold_label"].values
        y_pred = eval_df["sent_label"].values

        print("ACCURACY:", accuracy_score(y_true, y_pred))
        report = classification_report(y_true, y_pred, labels=["positivo","negativo","neutro"], output_dict=True, zero_division=0)
        display(pd.DataFrame(report).T)

        cm = confusion_matrix(y_true, y_pred, labels=["positivo","negativo","neutro"])
        cm_df = pd.DataFrame(cm, index=["gold_pos","gold_neg","gold_neu"], columns=["pred_pos","pred_neg","pred_neu"])
        display(cm_df)

        # Brier score (se tiver probabilidades)
        prob_cols = ["sent_p_positivo","sent_p_negativo","sent_p_neutro"]
        if all(c in eval_df.columns for c in prob_cols):
            P = eval_df[prob_cols].to_numpy(dtype=float)
            classes = ["positivo","negativo","neutro"]
            briers = []
            for i, cls in enumerate(classes):
                y_bin = (y_true == cls).astype(int)
                briers.append(brier_score_loss(y_bin, P[:, i]))
            display(pd.DataFrame({"classe": classes, "brier": briers}))

        # Cobertura × acurácia por threshold de confiança
        if "sent_conf" in eval_df.columns:
            def acc_at(t):
                sub = eval_df[eval_df["sent_conf"] >= t]
                if sub.empty: return np.nan, 0
                return accuracy_score(sub["gold_label"], sub["sent_label"]), len(sub)
            ts = np.linspace(0.0, 0.95, 20)
            covdf = pd.DataFrame([
                {"threshold": float(t),
                 "accuracy": acc_at(float(t))[0],
                 "coverage": acc_at(float(t))[1]/len(eval_df)}
                for t in ts
            ])
            display(covdf)

            # gráfico simples (1 figura, sem cores específicas)
            import matplotlib.pyplot as plt
            plt.figure()
            plt.plot(covdf["threshold"], covdf["accuracy"], label="Accuracy")
            plt.plot(covdf["threshold"], covdf["coverage"], label="Coverage")
            plt.xlabel("Confidence threshold")
            plt.ylabel("Metric value")
            plt.title("Coverage & Accuracy vs. Confidence Threshold")
            plt.legend()
            plt.show()
    else:
        print("Sem rótulos humanos válidos em 'gold_label'; métricas de acurácia não foram calculadas.")
else:
    print("Coluna 'gold_label' não encontrada; exibidas apenas visões descritivas.")

,sent_label,count
0,negativo,40
1,positivo,24
2,neutro,5


,tipo,sent_label,count
0,ata,negativo,23
1,ata,neutro,4
4,mensal,positivo,24
2,mensal,negativo,17
3,mensal,neutro,1


,sent_label,count,mean,median,min,max
0,negativo,40,0.511221,0.509303,0.363790,0.759896
2,positivo,24,0.493318,0.482361,0.381392,0.662053
1,neutro,5,0.485584,0.480314,0.422176,0.530513


,yyyymm,sent_label,count
0,202201,positivo,1
1,202202,negativo,2
2,202203,neutro,1
3,202203,positivo,1
4,202205,negativo,1
5,202205,positivo,1
6,202206,negativo,1
7,202207,positivo,1
8,202208,negativo,1
9,202209,negativo,1


,doc,doc_src,tipo,yyyymm,sent_label,sent_conf,snippet
19,202302_estm,202302_mensal,mensal,202302,negativo,0.363790,
52,202412_estm,202412_mensal,mensal,202412,negativo,0.373515,
67,202509_estm,202509_mensal,mensal,202509,positivo,0.381392,
1,202202_estm,202202_mensal,mensal,202202,negativo,0.387029,
44,202407_estm,202407_mensal,mensal,202407,negativo,0.395016,
66,202508_estm,202508_mensal,mensal,202508,positivo,0.397181,
42,202406_estm,202406_mensal,mensal,202406,positivo,0.398514,
60,202505_estm,202505_mensal,mensal,202505,positivo,0.405451,
57,202503_estm,202503_mensal,mensal,202503,positivo,0.414476,
0,202201_estm,202201_mensal,mensal,202201,positivo,0.419371,


Coluna 'gold_label' não encontrada; exibidas apenas visões descritivas.
